### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [3]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [4]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [5]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [6]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Perfect! Just hop on our magic carpet – oh wait, it's grounded. How about a plane instead?"

### Let's make a local database of ticket prices

In [7]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [8]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)
save_city_price("Taipei", 1200)
save_city_price("Beijing", 1500)

In [9]:
# Method to get cities
def get_cities():
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT * FROM cities")
    rows = c.fetchall()    # ← fetch rows while connection is open
    conn.close()
    return rows


In [10]:
rows = get_cities()

for row in rows:
    print(row)

('london', 299.0)
('paris', 399.0)
('rome', 499.0)
('madrid', 550.0)
('barcelona', 580.0)
('berlin', 525.0)
('taipei', 1200.0)
('beijing', 1500.0)


In [11]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [12]:
print (get_city_price("Rome"))
print (get_city_price("Beijing"))

499.0
1500.0


In [13]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [14]:
async def chat_message_content():
    response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
    for inner_message in response.inner_messages:
        print(inner_message.content)
    return response.chat_message.content

In [15]:
result = await chat_message_content()
print (result)

[FunctionCall(id='call_vWuJLnTKfRuhDCQhPIAm0NEE', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_vWuJLnTKfRuhDCQhPIAm0NEE', is_error=False)]
Going to London? Great choice! Grab your umbrella and your roundtrip ticket for just $299. Don't forget to practice your "I'm not lost, just exploring" face!


In [16]:
message = TextMessage(content="I'd like to go to Taipei", source="user")
result = await chat_message_content()
print (result)

[FunctionCall(id='call_pnBxANtNtrEUfOmYSIYACisd', arguments='{"city_name":"Taipei"}', name='get_city_price')]
[FunctionExecutionResult(content='1200.0', name='get_city_price', call_id='call_pnBxANtNtrEUfOmYSIYACisd', is_error=False)]
Ah, Taipei! Home of bubble tea and night markets! Your roundtrip ticket is just $1200. A small price to pay for all the delicious dumplings you'll munch on! 🥢😄


In [17]:
message = TextMessage(content="I'd like to go to Beijing", source="user")
result = await chat_message_content()
print (result)

[FunctionCall(id='call_jZ9hgnJpz6HLDTOKp5XkIdsx', arguments='{"city_name":"Beijing"}', name='get_city_price')]
[FunctionExecutionResult(content='1500.0', name='get_city_price', call_id='call_jZ9hgnJpz6HLDTOKp5XkIdsx', is_error=False)]
Beijing, huh? You're in for a treat—great food and the Great Wall! A roundtrip ticket will cost you $1500. Just remember, "great" doesn't mean you can jump the wall! 🧱😂


In [18]:
print (message.content)

I'd like to go to Beijing
